In [1]:
import scanpy as sc
import os,sys,glob
import pandas as pd
import numpy as np
import anndata as ad
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import pandas as pd
import numpy as np
import decoupler as dc
import re

In [2]:
sc.settings.n_jobs = 30
sc.set_figure_params(figsize=(4,4.5), vector_friendly = True)
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

In [3]:
adata = sc.read_h5ad("Data/Output_240327_adata_scvi_random_sampleID.h5ad") # 1m

In [4]:
adata

AnnData object with n_obs × n_vars = 393060 × 49133
    obs: 'sampleID', 'donorID', 'batch', 'Age', 'Assay', 'Stage', 'Race', 'PMI', 'Hemisphere', 'Library', 'Brain Region', 'Dataset', 'Sex', 'Diagnosis', 'cluster_original', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'doublet_scores', 'predicted_doublets', '_scvi_batch', '_scvi_labels', 'leiden_0.6', 'cluster_annotated', 'Leiden', 'Cell Type', 'Sex_impute'
    uns: 'Brain Region_colors', 'Brain_Region_colors', 'Cell Type_colors', 'Stage_colors', 'cluster_annotated_colors', 'leiden_0.6_colors', 'log1p', 'ttest_pts', 'wilcoxon'
    obsm: 'X_scVI_sampleID', 'X_umap', '_scvi_extra_categorical_covs'
    layers: 'logcounts', 'scaled'

# Pseudobulk

In [5]:
adata.obs.columns

Index(['sampleID', 'donorID', 'batch', 'Age', 'Assay', 'Stage', 'Race', 'PMI',
       'Hemisphere', 'Library', 'Brain Region', 'Dataset', 'Sex', 'Diagnosis',
       'cluster_original', 'n_genes', 'n_genes_by_counts', 'total_counts',
       'total_counts_mt', 'pct_counts_mt', 'doublet_scores',
       'predicted_doublets', '_scvi_batch', '_scvi_labels', 'leiden_0.6',
       'cluster_annotated', 'Leiden', 'Cell Type', 'Sex_impute'],
      dtype='object')

In [6]:
# adata.obs['batch'] = adata.obs['batch'].astype("category")
# adata.obs['sampleID'] = adata.obs['sampleID'].astype("category")
# adata.obs['Sex'] = adata.obs['Sex'].astype("category")
# adata.obs['cluster_number'] = adata.obs['cluster_number'].astype("category")

In [7]:
# Get filtered pseudo-bulk profile
pdata = dc.get_pseudobulk(
    adata,
    sample_col='sampleID',
    groups_col='Leiden',
    layer='logcounts',
    mode='mean',
    min_cells=30,
    min_counts=1000
)
pdata

/home/ssun1116/anaconda3/envs/decoupler/lib/python3.11/site-packages/decoupler/utils_anndata.py:181: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cols = obs.groupby([sample_col, groups_col]).apply(lambda x: x.apply(lambda y: len(y.unique()) == 1)).all(0)
/home/ssun1116/anaconda3/envs/decoupler/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


AnnData object with n_obs × n_vars = 999 × 36004
    obs: 'sampleID', 'donorID', 'batch', 'Age', 'Assay', 'Stage', 'Race', 'PMI', 'Hemisphere', 'Library', 'Brain Region', 'Dataset', 'Sex', 'Diagnosis', '_scvi_batch', '_scvi_labels', 'leiden_0.6', 'cluster_annotated', 'Leiden', 'Cell Type', 'Sex_impute', 'psbulk_n_cells', 'psbulk_counts'
    layers: 'psbulk_props'

In [8]:
pdata.obs["Age"] = pdata.obs["Age"].astype(str)
pdata.obs["_scvi_batch"] = pdata.obs["_scvi_batch"].astype(str)
pdata.obs["_scvi_labels"] = pdata.obs["_scvi_labels"].astype(str)
pdata.obs["leiden_0.6"] = pdata.obs["leiden_0.6"].astype(str)

In [9]:
pdata.write_h5ad("Data/Pseudobulk_mincells_30_lognorm_mean_0327.h5ad")